In [1]:
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('./tools/')
from AlignFilter import *
from LocalGeo import *
from MapperTools import *
from DataProcess import *

/share/user_data/xianjunxing/.conda/envs/python_tda/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/share/user_data/xianjunxing/.conda/envs/python_tda/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/share/user_data/xianjunxing/.conda/en

In [2]:
# STIM step-1
## generate a shared low-dimensional embedding space

In [3]:
def STIM_LowDimSpace(clip_ts_ls,model):
    """
    Function to transform a list of time-series (TS) data into a low-dimensional space using UMAP.
    
    Parameters:
    clip_ts_ls: list of numpy.ndarray. each element:n_subj * ROIs * nTR time-series data arrays.
    **kwargs: Additional keyword arguments to be passed to the align_umap function.
    
    Returns:
    numpy.ndarray: low-dimensional embeddings for the input time-series data.
    """
    
    n_subs = clip_ts_ls[0].shape[0]  # Number of subjects (assumed to be the same across all clips)
    
    # Normalize the time-series data along the axis representing ROIs (regions of interest)
    ts_norm = []  # Initialize an empty list to store normalized time-series
    for ts_i in clip_ts_ls:
        # Scale each subject's time-series data along the ROI axis
        ts_i_norm = [scale(ts_i[i], axis=1) for i in range(n_subs)]
        ts_norm.append(np.array(ts_i_norm))  # Append the normalized data for each clip/movie
    # Select indices of movies/clips longer than 2 minutes and concatenate their time-series data
    ts_arr = np.dstack([ts_i for ts_i in ts_norm])  # Stack the selected clips along a new axis
    print('time-series shape =',ts_arr.shape)  # Print the shape of the concatenated time-series data
    
    # Transform the time-series data into 3-dimensional UMAP embeddings
    # Note: This step is computationally intensive and may take a long time to run
    sub_embeds = align_projection(ts_arr=ts_arr,model=model)
    print('embeddings shape =',sub_embeds.shape)  # Print the shape of the resulting UMAP embeddings
    
    return sub_embeds  # Return the UMAP embeddings

In [4]:
# STIM step-2
## generate individual-group shape graph, compute intersection TCM

In [5]:
def STIM_individual_mapper(sub_embeds, group_embeds=None, cover_n=12, cover_overlap=0.5, eps=0.7):
    """
    Function to map individual subject UMAP embeddings to a group-level UMAP and calculate sub-matrices of the Temporal Connectivity Matrix (TCM).
    
    Parameters:
    sub_embeds (numpy.ndarray): UMAP embeddings for individual subjects, shape (n_sub, n_TR, n_components).
    group_umap (numpy.ndarray or None): Optional group-level UMAP embeddings. If None, the mean of sub_umap is used. Default is None.
    cover_n (int): Number of intervals in the cover (resolution parameter for Mapper). Default is 12.
    cover_overlap (float): Fractional overlap of intervals in the cover. Default is 0.5.
    eps (float): Epsilon parameter for DBSCAN clustering. Default is 0.7.
    
    Returns:
    list: A list of sub-matrices of the Temporal Connectivity Matrix (TCM) for each subject.
    """
    
    # If group_umap is not provided, calculate it as the mean of individual subject embeddings
    if group_embeds is None:
        group_embeds = np.mean(sub_embeds, axis=0)
    
    # Normalize the group UMAP embeddings along the features (columns)
    group_embeds_scale = scale(group_embeds, axis=0)
    
    sub_score = []  # Initialize an empty list to store scores (not used in this code)
    sub_n = np.array(sub_embeds).shape[0]  # Number of subjects
    
    # Normalize each subject's UMAP embeddings
    sub_embeds_scale = [scale(sub_embeds[i], axis=0) for i in range(sub_n)]
    
    # Combine all subjects' embeddings and the group embedding into a single matrix
    embeds = np.vstack([sub_embeds_scale[i] for i in range(sub_n)] + [group_embeds_scale])
    g_size = int(group_embeds_scale.shape[0])  # Size of the group embedding (number of time points)
    
    # Initialize KeplerMapper
    mapper = KeplerMapper(verbose=0)
    
    # Fit the Mapper to the combined embeddings and project onto the first three components
    lens = mapper.fit_transform(embeds, projection=[0, 1, 2])
    
    # Extract the lens (projection) corresponding to the group-level UMAP
    group_lens = lens[g_size * sub_n:, :]
    group_tcm = []  # Initialize an empty list to store the group TCM (not used in this code)

    args_list = []  # Initialize a list to store arguments for parallel processing
    
    # Prepare the arguments for each subject for parallel processing
    for i in tqdm(range(sub_n)):
        # Extract the lens for the current subject
        sub_lens = lens[g_size * i:g_size * (i + 1), :]
        
        # Combine the group lens with the current subject lens
        lens_i = np.vstack([group_lens, sub_lens])
        
        # Combine the group embedding with the current subject embedding
        embeds_i = np.vstack([group_embeds_scale, sub_embeds_scale[i]])
        
        # Append the arguments to the list
        args_list.append([embeds_i, g_size, cover_n, cover_overlap, lens_i, eps])

    # Initialize a multiprocessing pool with the number of available CPU cores
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    
    # Process each subject in parallel to compute the sub-matrices of the TCM
    results = pool.map(process_sub_embeds, args_list)
    
    # Close the pool and wait for all processes to finish
    pool.close()
    pool.join()
    
    return results  # Return the list of sub-matrices of the TCM for each subject


In [6]:
# STIM step-3
## extract global topology and local geometry

In [7]:
def STIM_global_topology(indi_TCM):
    """
    Function to extract the global topology from individual Temporal Connectivity Matrices (TCM).
    
    Parameters:
    indi_TCM (numpy.ndarray): A 3D array of shape (n_subj, nTR, nTR) representing individual TCMs,
                              where n_subj is the number of subjects, and nTR is the number of time points.
    
    Returns:
    numpy.ndarray: A 2D array where each row contains the diagonal elements of the corresponding TCM,
                   representing the global topology for each subject.
    """
    # Extract the diagonal elements (global topology) from each individual's TCM
    glob_topo = np.array([_.diagonal() for _ in indi_TCM])
    
    return glob_topo  # Return the global topology for all subjects

def STIM_local_geometry(indi_TCM, template_TCM):
    """
    Function to analyze the local geometry of individual TCMs relative to a template TCM.
    
    Parameters:
    indi_TCM (numpy.ndarray): A 3D array of shape (n_subj, nTR, nTR) representing individual TCMs.
    template_TCM (numpy.ndarray): A 3D array of shape (template_subj, nTR, nTR) representing the template TCM,
                                  where template_subj is the number of template subjects.
    
    Returns:
    numpy.ndarray: A 2D array where each row contains the weighted clique scores for each segment (scene)
                   identified in the TCMs, representing the local geometry for each subject.
    """
    
    # Calculate the mean TCM across template subjects and normalize it
    template_TCM = template_TCM / max(template_TCM.flatten())  # Normalize the mean TCM by its maximum value
    
    # Find the best segmentation (scene transitions) in the mean TCM using a penalty search
    best_scene_tr, performance = search_best_param(template_TCM, method='tr_level', penalty=range(2, 60))
    
    clique_score = []  # Initialize a list to store the clique scores for each segment
    
    # Iterate over each identified scene transition (segment)
    for tr_i in best_scene_tr:
        a, b = tr_i  # Extract the start and end time points of the segment
        
        # Extract the corresponding submatrix (clique) from the individual TCMs
        clique_mtx = indi_TCM[:, a:b, a:b]
        
        # Symmetrize each clique by averaging it with its transpose
        clique_mtx_symm = np.array([(_.T + _) / 2 for _ in clique_mtx])
        
        # Calculate the weighted clique score for each symmetrized submatrix
        clique_score.append(weighted_clique_score(clique_mtx_symm))
    
    clique_score = np.array(clique_score)  # Convert the list of clique scores to a numpy array
    
    return clique_score,best_scene_tr  # Return the local geometry (clique scores) for all subjects

In [8]:
# generate shape graph
from kmapper import KeplerMapper, Cover
from sklearn.cluster import DBSCAN
from dyneusr import DyNeuGraph

def g_mapper_graph(low_d_embeds,cover_n=20,cover_overlap=0.4,eps=0.5):
    embeds = low_d_embeds
    mapper = KeplerMapper(verbose=0)
    lens = mapper.fit_transform(embeds,projection=[0,1,2])
    graph = mapper.map(lens, X=embeds, cover=Cover(cover_n, cover_overlap, limits=np.array([[0,1],[0,1],[0,1]])),clusterer=DBSCAN(eps=eps), )
    return graph

In [9]:
# edit to your data path
hcp_18clips_ts = load_npz('../data/Fig2/ts/hcp_18clips_ts_l5r5.npz')

In [10]:
# Remove clips with less than two minutes of data
hcp_13clips_ts = []
movie_ind = [0, 1, 2, 5, 6, 7, 9, 10, 11, 12, 14, 15, 16]
for i in movie_ind:
    hcp_13clips_ts.append(hcp_18clips_ts[i])

In [11]:
# Select a filter (here, we use UMAP)
umap_model = UMAP(n_components=3,n_neighbors=25,random_state=42,min_dist=0.01,metric='euclidean')

# Apply STIM to the data
hcp_13clips_umap = STIM_LowDimSpace(hcp_13clips_ts,umap_model)

# Compute group consensus
group_umap = hcp_13clips_umap.mean(axis=0)
graph_group = g_mapper_graph(low_d_embeds=scale(group_umap, axis=0), cover_n=12, cover_overlap=0.5, eps=0.7)

time-series shape = (170, 271, 2804)
embeddings shape = (170, 2804, 3)


In [13]:
# Compute group consensus
group_umap = hcp_13clips_umap.mean(axis=0)
graph_group = g_mapper_graph(low_d_embeds=scale(group_umap, axis=0), cover_n=12, cover_overlap=0.5, eps=0.7)

In [ ]:
# visualize shape graph
dG = DyNeuGraph(G=graph_group)
dG.visualize('your_path')

In [14]:
hcp_170subs_tcm = STIM_individual_mapper(hcp_13clips_umap)
print(np.array(hcp_170subs_tcm).shape)

100%|██████████| 170/170 [00:00<00:00, 23907.98it/s]


(170, 2804, 2804)


In [15]:
hcp_glob_topo = STIM_global_topology(hcp_170subs_tcm)
print(hcp_glob_topo.shape)

(170, 2804)


In [16]:
hcp_tcm_mov1 = np.array(hcp_170subs_tcm)[:,:243,:243]
hcp_local_geo, clip_events = STIM_local_geometry(indi_TCM=hcp_tcm_mov1,template_TCM=hcp_tcm_mov1.mean(axis=0))
print(hcp_local_geo.shape)

100%|██████████| 58/58 [00:04<00:00, 12.76it/s]

(9, 170)


In [17]:
clip_events

[(1, 35),
 (36, 58),
 (59, 79),
 (80, 120),
 (121, 130),
 (131, 152),
 (153, 164),
 (165, 181),
 (182, 242)]